
# The CHSH Game and Quantum Advantage

## 1. Rules of the Game
The **CHSH game** (Clauser–Horne–Shimony–Holt) involves two players, **Alice** and **Bob**, who cooperate against a referee:

1. The referee sends **random input bits**:
   - Alice receives $x \in \{0,1\}$.
   - Bob receives $y \in \{0,1\}$.
2. Each player outputs one bit:
   - Alice outputs $a \in \{0,1\}$.
   - Bob outputs $b \in \{0,1\}$.
3. **Winning condition**:
   $$
   a \oplus b = x \cdot y
   $$
   where $\oplus$ is XOR and $\cdot$ is AND.

> Constraint: Alice and Bob **cannot communicate** after receiving their inputs.

---

## 2. Classical Strategies
- Without entanglement, the best possible strategy achieves:
  $$
  P_{\text{win}}^{\text{classical}} \leq 0.75
  $$
- Example: if both always answer $0$, they win 3 out of 4 cases.

---

## 3. Quantum Strategy
By sharing a **maximally entangled state**:
$$
|\Phi^{+}\rangle = \tfrac{1}{\sqrt{2}} (|00\rangle + |11\rangle)
$$

Alice and Bob can outperform any classical strategy.

### Measurement Settings
- **Alice**:
  - If $x=0$: measure along $\sigma_Z$.
  - If $x=1$: measure along $\sigma_X$.
- **Bob**:
  - If $y=0$: measure along $\tfrac{\sigma_Z + \sigma_X}{\sqrt{2}}$.
  - If $y=1$: measure along $\tfrac{\sigma_Z - \sigma_X}{\sqrt{2}}$.

### Outcome Mapping
- Measurement results $\pm 1$ are mapped to bits $\{0,1\}$.

---

## 4. Quantum Winning Probability
With this setup:
$$
P_{\text{win}}^{\text{quantum}} = \cos^2\!\left(\tfrac{\pi}{8}\right) \approx 0.854
$$

This exceeds the classical maximum of $0.75$.

---

## 5. CHSH Inequality
The correlations are expressed as expectation values:
$$
E(x,y) = \langle A_x \otimes B_y \rangle
$$

The **CHSH parameter** is:
$$
S = E(0,0) + E(0,1) + E(1,0) - E(1,1)
$$

- **Classical bound**: $|S| \leq 2$.  
- **Quantum bound (Tsirelson)**: $|S| \leq 2\sqrt{2} \approx 2.828$.

---

## 6. Applications
- **Testing quantum nonlocality** (violations of Bell’s theorem).  
- **Device-independent quantum cryptography** (QKD without trusting hardware).  
- **Randomness generation** and certification.  
- **Self-testing of quantum devices**.  
- **Quantum networks**: verifying entanglement between distant nodes.  

---

## 7. Significance
The CHSH game is one of the clearest demonstrations that:
- Quantum mechanics allows correlations stronger than any classical theory.  
- These correlations enable **new technologies** in cryptography, communication, and computation.  


In [ ]:
# Simulate the CHSH game in Qiskit: quantum strategy vs classical baseline.

import math
import random
from collections import Counter

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# --------- Helpers ---------
def make_bell_pair_circuit():
    """Create |Φ+> = (|00> + |11>)/√2 on qubits (0,1)."""
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    return qc
print(make_bell_pair_circuit())
def add_measurement_in_xz_plane(qc, qubit, theta):
    """
    Measure along observable cos(theta) Z + sin(theta) X.
    Implement by rotating by R_y(theta) then measuring Z.
    (Because U = R_y(theta) => U† Z U = cos(theta) Z + sin(theta) X)
    """
    qc.ry(theta, qubit)
    qc.measure(qubit, qubit)  # measure Z into matching classical bit

def chsh_angles():
    """
    Angles (from Z toward X) for optimal CHSH strategy:
      Alice:  x=0 -> 0 (Z), x=1 -> π/2 (X)
      Bob:    y=0 -> +π/4,   y=1 -> -π/4
    """
    A = {0: 0.0, 1: math.pi / 2}
    B = {0: math.pi / 4, 1: -math.pi / 4}
    return A, B

def build_measurement_circuit(x, y):
    """
    Build a 2-qubit circuit:
      - Prepare |Φ+>
      - Alice measures qubit 0 at angle A_x
      - Bob   measures qubit 1 at angle B_y
    """
    A, B = chsh_angles()
    qc = make_bell_pair_circuit()
    # Add measurements (put measures last)
    add_measurement_in_xz_plane(qc, 0, A[x])
    add_measurement_in_xz_plane(qc, 1, B[y])
    return qc

def win_indicator(x, y, a, b):
    """Return 1 if a XOR b == x AND y, else 0."""
    return int((a ^ b) == (x & y))

def counts_to_probabilities(counts, shots):
    return {k: v / shots for k, v in counts.items()}

def correlator_E_from_counts(counts, xy, shots):
    """
    E(x,y) = ⟨A_x * B_y⟩ where outcomes are mapped {0->+1, 1->-1}.
    For two-bit strings "ab": a is Alice (c0), b is Bob (c1).
    """
    E = 0.0
    for bitstr, c in counts.items():
        # Qiskit returns bitstrings 'ba' by default if using measure(q, q) it aligns; we mapped cbit=qubit.
        # Here, we measured qubit 0 -> c0, qubit 1 -> c1, so bitstring is "b a"? No:
        # When measuring into specific classical bits equal to qubit indices, Qiskit prints with most-significant
        # classical bit last. To avoid confusion, extract via integers directly:
        # We'll parse little-endian: bit 0 is the rightmost char (c0), bit 1 is the next (c1).
        # Safer approach: parse as integer bits:
        # But 'counts' keys are strings like '00' with bit 0 on the right.
        a_bit = int(bitstr[-1])  # c0 (Alice)
        b_bit = int(bitstr[-2])  # c1 (Bob), because two-bit string length=2
        a_val = +1 if a_bit == 0 else -1
        b_val = +1 if b_bit == 0 else -1
        E += a_val * b_val * (c / shots)
    return E

# --------- Simulation ---------
def simulate_chsh(shots_per_setting=10000, seed=1234):
    """
    Run the four input pairs (x,y) uniformly.
    Returns: win_prob, S_value, per_setting_stats
    """
    rng = random.Random(seed)
    sim = AerSimulator(seed_simulator=seed)

    per_setting = {}
    total_wins = 0
    total_shots = 0

    # Build, transpile, and run 4 circuits (xy in {00,01,10,11})
    circuits = []
    settings = []
    for x in (0, 1):
        for y in (0, 1):
            qc = build_measurement_circuit(x, y)
            qc = transpile(qc, sim, optimization_level=1, seed_transpiler=seed)
            circuits.append(qc)
            settings.append((x, y))

    job = sim.run(circuits, shots=shots_per_setting)
    results = job.result()

    E = {}
    for idx, (x, y) in enumerate(settings):
        counts = results.get_counts(idx)
        shots = sum(counts.values())
        probs = counts_to_probabilities(counts, shots)

        # Compute wins for this (x,y)
        wins_xy = 0
        for bitstr, p in probs.items():
            a = int(bitstr[-1])  # Alice bit
            b = int(bitstr[-2])  # Bob bit
            wins_xy += p * win_indicator(x, y, a, b)

        total_wins += wins_xy * shots
        total_shots += shots

        # Correlator
        E[(x, y)] = correlator_E_from_counts(counts, (x, y), shots)

        per_setting[(x, y)] = {
            "counts": dict(counts),
            "win_rate_xy": wins_xy,
            "E_xy": E[(x, y)],
        }

    win_prob = total_wins / total_shots

    # CHSH S = E(0,0) + E(0,1) + E(1,0) - E(1,1)
    S = E[(0, 0)] + E[(0, 1)] + E[(1, 0)] - E[(1, 1)]

    return win_prob, S, per_setting

def classical_baseline():
    """
    Best classical winning probability for CHSH is 0.75.
    (E.g., always output a=0, b=0)
    """
    return 0.75

# --------- Run ----------
if __name__ == "__main__":
    win_q, S_q, details = simulate_chsh(shots_per_setting=20000, seed=42)
    print("=== CHSH Quantum Strategy (Qiskit/Aer) ===")
    print(f"Winning probability  ≈ {win_q:.4f} (theory: ~0.8536)")
    print(f"CHSH S-value        ≈ {S_q:.4f} (Tsirelson bound: 2√2 ≈ {2*math.sqrt(2):.4f})")
    print("\nPer-setting stats (x,y): win_rate_xy, E_xy")
    for (x, y), s in sorted(details.items()):
        print(f"  ({x},{y}): win≈{s['win_rate_xy']:.4f}, E≈{s['E_xy']:.4f}")

    print("\n=== Classical Baseline ===")
    print(f"Best classical win probability = {classical_baseline():.2f}")


     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
c: 2/══════════
               
=== CHSH Quantum Strategy (Qiskit/Aer) ===
Winning probability  ≈ 0.8547 (theory: ~0.8536)
CHSH S-value        ≈ 2.8377 (Tsirelson bound: 2√2 ≈ 2.8284)

Per-setting stats (x,y): win_rate_xy, E_xy
  (0,0): win≈0.8509, E≈0.7018
  (0,1): win≈0.8565, E≈0.7131
  (1,0): win≈0.8579, E≈0.7157
  (1,1): win≈0.8536, E≈-0.7071

=== Classical Baseline ===
Best classical win probability = 0.75
